mAP
===

mAP是目标检测领域的一个常用指标

# 1.AP
对于AP来说，我们需要先了解[Accuracy，Precision，Recall](../../01.基础知识/09.其它/02.模型评估指标.ipynb)，[IOU](03.其它.01.IOU.ipynb)

## 1.1.Recall@TopK与Precision@TopK
说到Recall的时候，有时候会提及TopK的Recall和Precision。下面是一个小例子，假设有一个目标检测数据集，这个数据集中7张图片。下面是检测结果：

![image](Images/03_03_001.jpg)

上面七张图每张图片的检测结果均已标出，绿色是Ground Truth，红色是检测到的对象(总共24个，A~Y)。对上面的所有检测结果按照confidence排名统计出一个表如下，($IOU \geq 0.5$则被判定为正样本)：

![image](Images/03_03_002.jpg)

第三列表示预测的置信度(Confidence)排名，第四列表示当前目标框预测正确与否，倒数第一，第二列实际上是TopK的Precision和Recall。

那么Top1时，预测了一个正样本，且预测正确，所以Precision=1.0。同理可以算出Recall=0.0666，即总共15个正样本，却只预测出来1个，召回比例就是0.0666。同理TopK的Precision和Recall均可以计算。

## 1.2.AP
显然，上述例子中，随着K的增大，Recall和Precision会一增一减。那么以Recall为横轴，Precision为纵轴，就可以画出一条PR曲线如下

![image](Images/03_03_003.jpg)

上图PR曲线下的面积就定义为AP，即：$AP=\int_0^1p(r)dr$

## 1.3.PASCAL VOC 2008的评测指标-插值AP
![image](Images/03_03_004.jpg)

在PASCAL VOC 2008中，在计算AP之前会对上述曲线进行平滑.通过平滑策略，上面蓝色的PR曲线就变成了红色的虚线了。平滑的好处在于，平滑后的曲线单调递减，不会出现摇摆的情况。

这样的话，随着Recall的增大，Precision逐渐降低，这才是符合逻辑的。实际计算时，对平滑后的Precision曲线进行均匀采样出11个点（每个点间隔0.1），然后计算这11个点的平均Precision。具体如下：

![image](Images/03_03_005.jpg)

$$
\begin{split}
AP &=\frac{1}{11} \times (AP_r(0) + AP_r(0.1)+...+AP_r(1.0)) \\
&=\frac{1}{11} \times (1 \times 0.666 + 0.4285 + 0.4285 + 0.4285 + 0 + 0 + 0 + 0 + 0 + 0) \\
&=26.84\%
\end{split}
$$

这种计算方法叫做插值AP(Interpolated AP)。对于PASCAL VOC有20个类别，那么mAP就是对20个类别的AP进行平均。

## 1.4.PASCAL 2010-2012的评测指标-AUC曲线
上述11点插值的办法由于插值点数过少，容易导致结果不准。一个解决办法就是内插所有点。所谓内插所有点，其实就是对上述平滑之后的曲线算曲线下面积

![image](Images/03_03_006.jpg)

这样计算之所以会更准确一点，可以这么看！原先11点采样其实算的是曲线下面积的近似，具体近似办法是：

取10个宽为0.1，高为Precision的小矩形的面积平均。现在这个则不然，现在这个算了无数个点的面积平均，所以结果要准确一些。在本例中
$$
\begin{split}
A1&=(0.0666-0) \times 1 = 0.0666 \\
A2&=(0.1333-0.0666) \times 0.06666 = 0.04446222 \\
A3&=(0.4-0.1333) \times 0.4285 = 0.11428095 \\
A4&=(0.4666-0.4) \times 0.3043=0.02026638 \\
AP&=A1+A2+A3+A4 \\
&=0.0666 + 0.04446222 + 0.11428095 + 0.02026638 \\
&=0.24560955 \\
&\approx 24.56\%
\end{split}
$$